# Building a Custom YOLOv4 Object Detector for Identifying Images with Labels in any given Image.
\- By [Karan Jagtiani](https://www.linkedin.com/in/karanjagtiani/)

## Step 1: Preparing the files.
### 1. Training Images
I have used the 50 images provided in the drive link below and have labelled them with [LabelImg](https://github.com/tzutalin/labelImg).
### 2. Testing Images
I have used the 7 total images provided in the drive link below.
### 3. Yolo Files
i) Yolov4 Configuration File (.cfg) [Link](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.cfg)  
Make the following changes to suit your needs, these are my changes to the following fields:  
```
width: 416  
height: 416  
// The above can be increased to 608x608 for better accuracy (but longer computation time).  
max_batches=2000  
steps=1600,1800  
classes=1   
```

ii) obj.data
```
classes = 1
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/Shunya/backup/
```

iii) obj.names (File with the classes seperated by line breaks)
```
object
```

iv) Yolov4 Pre-trained model file  
[Link](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137)

v) Renaming all training images according to a particular convention.  
- Example: 00000001.jpg  
- This script is also provided in the drive.

vi) Add all the images with their labels to a .rar file and upload to your desired folder in your drive.

## Step 2: Training the model.
- Go throught all the code blocks one by one in order to get your model trained.

## Finishing Up
1) Download and configure the files according to your needs and in the folder structure you want and upload it to your drive. Then change the URLs mentinoned in code blocks below according to your requirements.


2) All the files mentioned above that were used in my case are provided in this Drive Folder for reference.
[Drive Link](https://drive.google.com/drive/folders/1gCdWj3GLAYmT48m_W8HExjCEagUiyQ_s?usp=sharing)

In [ ]:
# Link your Google Drive with this collab instance.
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Create a symbolic link to access files from drive easily
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
# Check for the NVIDIA GPU information
!nvidia-smi

In [ ]:
# Cloneing the 'darknet' object detection model
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Compile darknet
!make

In [ ]:
# List out the folders in your drive.
!ls /mydrive

In [ ]:
# Copy the files needed for the configuration of training from drive to the VM.
!cp /mydrive/Shunya/yolov4_custom_custom.cfg ./cfg
!cp /mydrive/Shunya/obj.names ./data
!cp /mydrive/Shunya/obj.data  ./data

In [ ]:
# Get the labelled images rar file from the drive.
!cp /mydrive/Shunya/images.rar ./data
!mkdir data/images
!unrar e ./data/images.rar -d data/images

In [ ]:
"""Verify All Images Before Training with OpenCV"""
import os
import cv2

imagesPath = 'data/images'

count = 0
total = 0
print("Testing Images with Open CV...")
for dirname in os.listdir(imagesPath):
    if(dirname.endswith(".png") or dirname.endswith(".jpg") or dirname.endswith(".jpeg")):
        delete = False
        finalDir = imagesPath + "/" + dirname
        try:
            image = cv2.imread(finalDir)
            if image is None:
                delete = True
                count += 1
            #else:
            #  print(dirname + " Successfully Opened.")
        except:
            delete = True
            count += 1
        if delete:
            print("Deleting: " + finalDir)
            os.remove(finalDir)
            os.remove(finalDir.split(".")[0]+".txt")
        total += 1

print(count, "Images were not readable.")
print("Deleted", count, "Images out of total", total, "Images.")
print(total-count, "Images Remaining.")

In [ ]:
""" Deleting those files which dont have a corresponding label file or a image file. """
listDir = os.listdir(imagesPath)
for dirname in listDir:
    i = 0
    x = dirname.split(".")[0]
    for inner in listDir:
        if(x == inner.split(".")[0]):
            extension = inner.split(".")[1]
            if(extension == 'png' or extension == 'jpg' or extension == 'jpeg' or extension  == 'txt'):
                i += 1
                if(i == 2):
                    break
    if(i < 2):
        finalDir = imagesPath + "/" + dirname
        print("Removing:", finalDir + " since it doesnt have its correspoding file.")
        os.remove(finalDir)

**Prepare for training**

In [ ]:
import glob
images_list = glob.glob("data/images/*.png") + glob.glob("data/images/*.jpg") + glob.glob("data/images/*.jpeg")
print(images_list)
print(len(images_list))

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!rm yolov4_custom_custom_last.weights

In [ ]:
# Download weights yolov4.conv.137
!cp /mydrive/Shunya/yolov4.conv.137 ./
# !cp /mydrive/Shunya/backup/yolov4_custom_custom_last.weights ./

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov4_custom_custom.cfg yolov4.conv.137 -dont_show
# !./darknet detector train data/obj.data cfg/yolov4_custom_custom.cfg yolov4_custom_custom_last.weights -dont_show

Once the model is done training, you will have the model saved in your drive in the 'backup' folder with the name: model_final.weights

## Happy Detecting!